In [6]:
import pandas as pd
import numpy as np

# Data parsing
import scrapy
import requests as rs
import socks
import socket
import time
import concurrent.futures
from bs4 import BeautifulSoup as BS
from tqdm import tqdm
from fake_useragent import UserAgent
from joblib import Parallel, delayed
from itertools import chain
socks.set_default_proxy(socks.SOCKS5, "localhost", 9150)
socket.socket = socks.socksocket

In [8]:
test = pd.read_csv('./data/test.csv')
test['price'] = np.nan
test.head(2)

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,price
0,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,синий,NaN,"Все автомобили, представленные в продаже, прох...",1.2 LTR,105 N12,"{""engine-proof"":true,""tinted-glass"":true,""airb...",бензин,...,роботизированная,EUROPEAN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,NaN
1,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,чёрный,NaN,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6 LTR,110 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,...,механическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,NaN


In [10]:
test.columns

Index(['bodyType', 'brand', 'car_url', 'color', 'complectation_dict',
       'description', 'engineDisplacement', 'enginePower', 'equipment_dict',
       'fuelType', 'image', 'mileage', 'modelDate', 'model_info', 'model_name',
       'name', 'numberOfDoors', 'parsing_unixtime', 'priceCurrency',
       'productionDate', 'sell_id', 'super_gen', 'vehicleConfiguration',
       'vehicleTransmission', 'vendor', 'Владельцы', 'Владение', 'ПТС',
       'Привод', 'Руль', 'Состояние', 'Таможня', 'price'],
      dtype='object')

In [11]:
page_url = 'https://auto.ru/moskva/cars/{brand}/{year}-year/all'
headers = {'User-Agent': UserAgent().Chrome}
years = range(1960, 2021)

# Some functions to get the necessary urls
def get_brands():
    """
    This function returns all
    car brands from auto.ru
    """
    url = 'https://auto.ru/catalog/cars/'
    marks_list = []
    response = rs.get(url)
    response.encoding = 'utf-8'
    soup = BS(response.text, 'html.parser')
    all_brand_tags = soup.find('div', class_='search-form-v2-list search-form-v2-list_invisible search-form-v2-list_type_all clearfix')
    brand_columns = all_brand_tags.find_all('div', class_='search-form-v2-list__text-item')
    brand_list = [brand.a['href'].split('/')[-2] for brand in brand_columns]
    
    return brand_list


def url_generator():
    """
    This function returns all
    the urls for each year
    and car brand
    """
    all_pages = []
    for brand in get_brands():
        for year in years:
            all_pages.append(f'https://auto.ru/moskva/cars/{brand}/{year}-year/all/')

    return all_pages


def get_total_pages(url):
    """
    This function returns
    the last possible page 
    number for each car offers
    """
    response = rs.get(url, headers=headers)
    response.encoding = 'utf-8'
    pages = BS(response.text, 'html.parser').find_all('a',class_="Button Button_color_whiteHoverBlue Button_size_s Button_type_link Button_width_default ListingPagination-module__page")
    try:
        last_page = int(pages[-1].text)
        return last_page
    except:
        return 1
    
    
def get_pages(url):
    """
    Returns all the pages
    we need to scrape
    """
    pages_list = []
    last_page = get_total_pages(url)
    for page_i in range(1, last_page + 1):
        pages_list.append(f'{url}?page={str(page_i)}&output_type=list')
        
    return pages_list

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "/Users/bias/opt/anaconda3/lib/python3.7/site-packages/socks.py", line 787, in connect
    super(socksocket, self).connect(proxy_addr)
ConnectionRefusedError: [Errno 61] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/bias/opt/anaconda3/lib/python3.7/urllib/request.py", line 1319, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/Users/bias/opt/anaconda3/lib/python3.7/http/client.py", line 1252, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Users/bias/opt/anaconda3/lib/python3.7/http/client.py", line 1298, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Users/bias/opt/anaconda3/lib/python3.7/http/client.py", line 1247, in endheaders
    self

FakeUserAgentError: Maximum amount of retries reached